In [1]:
import os
import time
import logging
import argparse

import cv2
import numpy as np
import torch
import torch.backends.cudnn as cudnn
import torch.nn.functional as F
import torch.nn.parallel
import torch.utils.data
import torch.nn as nn

from model.pspnet import PSPNet, DeepLabV3
from util import dataset, transform, config
from PIL import Image
# from util.util import AverageMeter, intersectionAndUnion, check_makedirs, colorize

cv2.ocl.setUseOpenCL(False)

/cfarhomes/psando/miniconda3/envs/seg/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
from torchvision.utils import make_grid

plt.rcParams["savefig.bbox"] = 'tight'

def show(imgs, figsize=(10,10), save_filename=None):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fix, axs = plt.subplots(ncols=len(imgs), squeeze=False, figsize=figsize)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = transforms.functional.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])
    if save_filename:
        fix.savefig(save_filename, dpi=300)

In [5]:
# Model
from model.mixbn import MixBatchNorm2d

args = config.load_cfg_from_cfg_file('config/paper/voc2012/voc2012_pspnet50.yaml')

base_save_path = os.path.join('/vulcanscratch/psando/semseg_experiments', 'train_advprop_psp')
args.model_path = os.path.join(os.path.join(base_save_path, 'model'), 'train_epoch_50.pth')
args.save_path = os.path.join(base_save_path, 'result')

batch_norm = MixBatchNorm2d

model = torch.nn.DataParallel(PSPNet(layers=args.layers, classes=args.classes, zoom_factor=args.zoom_factor, pretrained=False, BatchNorm=batch_norm))
model = model.cuda()
cudnn.benchmark = True
if os.path.isfile(args.model_path):
    print("=> loading checkpoint '{}'".format(args.model_path))
    checkpoint = torch.load(args.model_path)
    model.load_state_dict(checkpoint['state_dict'], strict=True)
    print("=> loaded checkpoint '{}'".format(args.model_path))
else:
    raise RuntimeError("=> no checkpoint found at '{}'".format(args.model_path))

=> loading checkpoint '/vulcanscratch/psando/semseg_experiments/train_advprop_psp/model/train_epoch_50.pth'
=> loaded checkpoint '/vulcanscratch/psando/semseg_experiments/train_advprop_psp/model/train_epoch_50.pth'


In [8]:
model = model.eval()

In [12]:
model.module.layer0[1]

MixBatchNorm2d(
  64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
  (aux_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [18]:
model.module.layer0[1].weight

Parameter containing:
tensor([1.9941e-01, 2.4484e-01, 1.6613e-06, 2.1719e-06, 5.3578e-05, 1.7431e-01,
        4.3489e-01, 2.2361e-08, 2.5743e-01, 1.5029e-01, 1.8021e-01, 4.5049e-01,
        9.0471e-02, 8.0701e-02, 1.1520e-01, 8.5650e-07, 2.4974e-01, 4.3030e-01,
        7.0187e-06, 5.9870e-02, 1.2643e-06, 1.8235e-01, 1.6881e-01, 2.5544e-01,
        2.4502e-01, 3.5655e-08, 1.6903e-01, 2.7256e-01, 2.0809e-01, 2.5900e-01,
        2.1319e-08, 9.4099e-02, 1.7858e-01, 2.2511e-01, 1.7098e-01, 6.1804e-07,
        7.0976e-06, 1.5933e-01, 1.4840e-01, 1.5984e-01, 2.3096e-01, 8.0616e-06,
        3.3085e-01, 1.6960e-01, 1.6717e-01, 4.2818e-05, 1.1963e-01, 2.9062e-07,
        2.9910e-01, 1.2965e-01, 2.2730e-01, 6.2015e-06, 7.0814e-09, 1.4086e-01,
        1.5904e-01, 1.5913e-01, 2.8527e-01, 1.9741e-01, 1.5335e-04, 2.0480e-01,
        2.1538e-01, 2.4562e-01, 2.2961e-01, 8.8388e-08], device='cuda:0',
       requires_grad=True)

In [21]:
model.module.layer0[1].aux_bn.weight

Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0',
       requires_grad=True)